# **Camada Gold (Business / Aggregated Layer)**

Dados prontos para consumo, com agregações, sumarizações e cálculos.

Normalmente estruturados para atender demandas específicas do negócio.

Utilizado por dashboards, relatórios e aplicações finais.

In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_timbauvas")
display(df_silver)

In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_lcb_contemporaneo")
display(df_silver)